In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('crime-housing-austin-2015.csv')

In [33]:
# Convert dollar and percent values to floats
for col in df.select_dtypes(include=['object']):
    if '$' in df[col].iloc[0] or '%' in df[col].iloc[0]:
        df[col] = df[col].str.replace('\$|%', '').astype('float')

In [40]:
df.describe()

,Key,Council_District,Zip_Code_Crime,Census_Tract,X_Coordinate,Y_Coordinate,Zip_Code_Housing,Populationbelowpovertylevel,Medianhouseholdincome,Non-WhiteNon-HispanicorLatino,...,Rentalunitsaffordabletoaverageteacher,Ownerunitsaffordabletoaverageteacher,Rentalunitsaffordabletoaveragetechworker,Ownerunitsaffordabletoaveragetechworker,Changeinpercentageofpopulationbelowpoverty2000-2012,Changeinmedianrent2000-2012,Changeinmedianhomevalue2000-2012,Percentageofhomeswithin1/4-mioftransitstop,Averagemonthlytransportationcost,Percentageofhousingandtransportationcoststhatistransportation-related
count,3.857300e+04,38401.000000,38414.000000,38414.000000,3.798100e+04,3.798100e+04,37285.000000,37444.000000,37444.000000,37444.000000,...,37444.000000,37168.000000,37444.000000,37168.000000,37444.000000,37444.000000,37444.00000,37444.000000,37444.000000,37444.000000
mean,1.787442e+10,5.223458,78732.853777,72.426942,3.115612e+06,1.008005e+07,78734.385705,23.101458,50354.879367,15.954065,...,72.864331,37.092337,98.141304,71.712548,84.269336,36.528523,75.22073,66.183527,654.619886,37.749920
std,1.564763e+10,2.805340,25.877405,686.752321,1.508935e+04,2.857887e+04,22.619026,12.974184,20826.979519,7.035044,...,15.656461,28.642121,3.287924,29.073819,83.205696,25.092500,38.97744,26.165708,82.995217,5.806746
min,2.015103e+08,1.000000,78613.000000,1.010000,3.040396e+06,1.001173e+07,78617.000000,1.000000,11917.000000,4.000000,...,8.000000,0.000000,71.000000,6.000000,-20.000000,0.000000,-37.00000,0.000000,433.000000,23.000000
25%,2.015186e+10,3.000000,78719.000000,15.040000,3.106999e+06,1.005776e+07,78721.000000,16.000000,39593.000000,9.000000,...,67.000000,13.000000,99.000000,45.000000,33.000000,25.000000,49.00000,59.000000,629.000000,35.000000
50%,2.015500e+10,5.000000,78744.000000,18.230000,3.116494e+06,1.007645e+07,78744.000000,24.000000,41869.000000,16.000000,...,76.000000,28.000000,99.000000,86.000000,60.000000,28.000000,60.00000,76.000000,668.000000,40.000000
75%,2.015503e+10,8.000000,78752.000000,23.040000,3.126048e+06,1.010246e+07,78753.000000,29.000000,65672.000000,20.000000,...,83.000000,69.000000,100.000000,96.000000,128.000000,40.000000,93.00000,80.000000,708.000000,43.000000
max,2.015153e+12,10.000000,78759.000000,9800.000000,3.164242e+06,1.015923e+07,78759.000000,66.000000,127726.000000,36.000000,...,100.000000,87.000000,100.000000,100.000000,1242.000000,115.000000,207.00000,100.000000,865.000000,47.000000


In [41]:
df.columns

Index(['Key', 'Council_District', 'Highest_Offense_Desc',
       'Highest_NIBRS_UCR_Offense_Description', 'Report_Date', 'Location',
       'Clearance_Status', 'Clearance_Date', 'District', 'Zip_Code_Crime',
       'Census_Tract', 'X_Coordinate', 'Y_Coordinate', 'Zip_Code_Housing',
       'Populationbelowpovertylevel', 'Medianhouseholdincome',
       'Non-WhiteNon-HispanicorLatino', 'HispanicorLatinoofanyrace',
       'Populationwithdisability', 'Unemployment',
       'Largehouseholds(5+members)',
       'Homesaffordabletopeopleearninglessthan$50000',
       'Rentalsaffordabletopeopleearninglessthan$25000',
       'Rent-restrictedunits', 'HousingChoiceVoucherholders', 'Medianrent',
       'Medianhomevalue', 'Percentageofrentalunitsinpoorcondition',
       'Percentchangeinnumberofhousingunits2000-2012',
       'Ownerunitsaffordabletoaverageretail/serviceworker',
       'Rentalunitsaffordabletoaverageretail/serviceworker',
       'Rentalunitsaffordabletoaverageartist',
       'Ownerunits

In [57]:
for column in df.corr():
    print(column, df[column].max())

Key 2015153441365
Council_District 10.0
Zip_Code_Crime 78759.0
Census_Tract 9800.0
X_Coordinate 3164242.0
Y_Coordinate 10159232.0
Zip_Code_Housing 78759.0
Populationbelowpovertylevel 66.0
Medianhouseholdincome 127726.0
Non-WhiteNon-HispanicorLatino 36.0
HispanicorLatinoofanyrace 77.0
Populationwithdisability 17.0
Unemployment 16.0
Largehouseholds(5+members) 28.0
Homesaffordabletopeopleearninglessthan$50000 93.0
Rentalsaffordabletopeopleearninglessthan$25000 41.0
Rent-restrictedunits 47.0
HousingChoiceVoucherholders 20.0
Medianrent 2000.0
Medianhomevalue 621900.0
Percentageofrentalunitsinpoorcondition 2.2
Percentchangeinnumberofhousingunits2000-2012 239.0
Ownerunitsaffordabletoaverageretail/serviceworker 50.0
Rentalunitsaffordabletoaverageretail/serviceworker 39.0
Rentalunitsaffordabletoaverageartist 76.0
Ownerunitsaffordabletoaverageartist 50.0
Rentalunitsaffordabletoaverageteacher 100.0
Ownerunitsaffordabletoaverageteacher 87.0
Rentalunitsaffordabletoaveragetechworker 100.0
Ownerunits